# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
U. Dudzeviciute  ->  U. Dudzeviciute  |  ['U. Dudzeviciute']
Arxiv has 58 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.14189
extracting tarball to tmp_2411.14189... done.
Retrieving document from  https://arxiv.org/e-print/2411.14231


extracting tarball to tmp_2411.14231...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.14271


extracting tarball to tmp_2411.14271...

 done.


Found 142 bibliographic references in tmp_2411.14271/paper.bbl.
Issues with the citations
syntax error in line 291: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.14271-b31b1b.svg)](https://arxiv.org/abs/2411.14271) | **The properties of the interstellar medium in dusty, star-forming galaxies at $z \sim 2-4$: The shape of the CO spectral line energy distributions**  |
|| D. J. Taylor, et al. -- incl., <mark>U. Dudzeviciute</mark> |
|*Appeared on*| *2024-11-22*|
|*Comments*| *Accepted for publication in MNRAS; 17 pages, 7 figures*|
|**Abstract**|            The molecular gas in the interstellar medium (ISM) of star-forming galaxy populations exhibits diverse physical properties. We investigate the $^{12}$CO excitation of twelve dusty, luminous star-forming galaxies at $z \sim 2-4$ by combining observations of the $^{12}$CO from $J_{\rm up} = 1$ to $J_{\rm up} = 8$. The spectral line energy distribution (SLED) has a similar shape to NGC 253, M82, and local ULIRGs, with much stronger excitation than the Milky Way inner disc. By combining with resolved dust continuum sizes from high-resolution $870$-$\mu$m ALMA observations and dust mass measurements determined from multi-wavelength SED fitting, we measure the relationship between the $^{12}$CO SLED and probable physical drivers of excitation: star-formation efficiency, the average intensity of the radiation field $\langle U\rangle$, and the star-formation rate surface density. The primary driver of high-$J_{\rm up}$ $^{12}$CO excitation in star-forming galaxies is star-formation rate surface density. We use the ratio of the CO($3-2$) and CO($6-5$) line fluxes to infer the CO excitation in each source and find that the average ratios for our sample are elevated compared to observations of low-redshift, less actively star-forming galaxies and agree well with predictions from numerical models that relate the ISM excitation to the star-formation rate surface density. The significant scatter in the line ratios of a factor $\approx 3$ within our sample likely reflects intrinsic variations in the ISM properties which may be caused by other effects on the excitation of the molecular gas, such as cosmic ray ionization rates and mechanical heating through turbulence dissipation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.14189-b31b1b.svg)](https://arxiv.org/abs/2411.14189) | **Reevaluating $H_0$ Tension with Non-Planck CMB and DESI BAO Joint Analysis**  |
|| Y.-H. Pang, <mark>X. Zhang</mark>, Q.-G. Huang |
|*Appeared on*| *2024-11-22*|
|*Comments*| *5 pages, 1 figure, 2 tables*|
|**Abstract**|            $H_0$ tension in the spatially flat $\Lambda$CDM model is reevaluated by employing three sets of non-Planck CMB data, namely WMAP, WMAP+ACT, and WMAP+SPT, in conjunction with DESI BAO data and non-DESI BAO datasets including 6dFGS, SDSS DR7, and SDSS DR16. Our analysis yields $H_0 = 68.86\pm 0.68~\mathrm{km\ s^{-1} Mpc^{-1}}$ with WMAP+DESI BAO, $H_0 = 68.72\pm 0.51~\mathrm{km\ s^{-1} Mpc^{-1}}$ with WMAP+ACT+DESI BAO, and $H_0 = 68.62\pm 0.52~\mathrm{km\ s^{-1} Mpc^{-1}}$ with WMAP+SPT+DESI BAO. The results of non-Planck CMB+DESI BAO exhibit a $3.4\sigma$, $3.7\sigma$, and $3.8\sigma$ tension with the SH0ES local measurement respectively which are around $1 \sigma$ lower in significance for the Hubble tension compared to Planck CMB+DESI BAO. Moreover, by combining DESI BAO data+non-Planck CMB measurements, we obtain a more stringent constraint on the Hubble constant compared to non-DESI BAO data+non-Planck CMB data, as well as reducing the significance of the Hubble tension.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.14231-b31b1b.svg)](https://arxiv.org/abs/2411.14231) | **Determining the absolute chemical abundance of nitrogen and sulfur in the quasar outflow of 3C298**  |
|| M. Dehghanian, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2024-11-22*|
|*Comments*| *Accepted for publication in A&A- 11 pages, 10 figures*|
|**Abstract**|            Context. Quasar outflows are key players in the feedback processes that influence the evolution of galaxies and the intergalactic medium. The chemical abundance of these outflows provides crucial insights into their origin and impact. Aims. To determine the absolute abundances of nitrogen and sulfur and the physical conditions of the outflow seen in quasar 3C298. Methods. We analyze archival spectral data from the Hubble Space Telescope (HST) for 3C298. We measure Ionic column densities from the absorption troughs and compare the results to photoionization predictions made by the Cloudy code for three different spectral energy distributions (SED), including MF87, UVsoft, and HE0238 SEDs. We also calculate the ionic column densities of excited and ground states of N iii to estimate the electron number density and location of the outflow using the Chianti atomic database. Results. The MF87, UVsoft, and HE0238 SEDs yield nitrogen and sulfur abundances at super-solar, solar, and sub-solar values, respectively, with a spread of 0.4 to 3 times solar. Additionally, we determined an electron number density of log(ne) greater than 3.3 cm-3, with the outflow possibly extending up to a maximum distance of 2.8 kpc. Conclusions. Our results indicate solar metallicity within a 60 percent uncertainty range, driven by variations in the chosen SED and photoionization models. This study underscores the importance of SEDs impact on determining chemical abundances in quasars outflows. These findings highlight the necessity of considering a wider range of possible abundances, spanning from sub solar to super solar values.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.14271/./images/comb_SLED_grid.png', 'tmp_2411.14271/./images/excitation_drivers_SigmaSFR.png', 'tmp_2411.14271/./images/excitation_drivers_U_SFE.png']
copying  tmp_2411.14271/./images/comb_SLED_grid.png to _build/html/
copying  tmp_2411.14271/./images/excitation_drivers_SigmaSFR.png to _build/html/
copying  tmp_2411.14271/./images/excitation_drivers_U_SFE.png to _build/html/
exported in  _build/html/2411.14271.md
    + _build/html/tmp_2411.14271/./images/comb_SLED_grid.png
    + _build/html/tmp_2411.14271/./images/excitation_drivers_SigmaSFR.png
    + _build/html/tmp_2411.14271/./images/excitation_drivers_U_SFE.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\thetable}{A\arabic{table}}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The properties of the interstellar medium in dusty, star-forming galaxies at $*ph z* \sim 2-4$: The shape of the CO spectral line energy distributions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.14271-b31b1b.svg)](https://arxiv.org/abs/2411.14271)<mark>Appeared on: 2024-11-22</mark> -  _Accepted for publication in MNRAS; 17 pages, 7 figures_

</div>
<div id="authors">

D. J. Taylor, et al. -- incl., <mark>U. Dudzeviciute</mark>

</div>
<div id="abstract">

**Abstract:** The molecular gas in the interstellar medium (ISM) of star-forming galaxy populations exhibits diverse physical properties.We investigate the $^{12}$ CO excitation of twelve dusty, luminous star-forming galaxies at $z \sim$ 2--4 by combining observations of the $^{12}$ CO from $J_{\rm up} = 1$ to $J_{\rm up} = 8$ .The spectral line energy distribution (SLED) has a similar shape to NGC 253, M82, and local ULIRGs, with much stronger excitation than the Milky Way inner disc.By combining with resolved dust continuum sizes from high-resolution $870$ - $\upmu$ m ALMA observations and dust mass measurements determined from multi-wavelength SED fitting, we measure the relationship between the $^{12}$ CO SLED and probable physical drivers of excitation: star-formation efficiency, the average intensity of the radiation field $\langle U\rangle$ , and the star-formation rate surface density. The primary driver of high- $J_{\rm up}$ $^{12}$ CO excitation in star-forming galaxies is star-formation rate surface density.We use the ratio of the CO(3--2) and CO(6--5) line fluxes to infer the CO excitation in each sourceand find that the average ratios for our sample are elevated compared to observations of low-redshift, less actively star-forming galaxies and agree well with predictions from numerical models that relate the ISM excitation to the star-formation rate surface density.The significant scatter in the line ratios of a factor $\approx 3$ within our sample likely reflects intrinsic variations in the ISM properties which may be caused by other effects on the excitation of the molecular gas, such as cosmic ray ionization rates and mechanical heating through turbulence dissipation.

</div>

<div id="div_fig1">

<img src="tmp_2411.14271/./images/comb_SLED_grid.png" alt="Fig5" width="100%"/>

**Figure 5. -** CO SLEDs for our sample SMGs. Individual fluxes measured from fits to the spectra are shown as circles with error-bars indicating their propagated uncertainties determined via a Monte Carlo method.
    _ Top left:_ Fluxes are normalised to the flux (or upper limits) of the CO(1--0) line and are coloured by their respective star-formation rate surface densities denoted by the colour bar (applicable to both top- and bottom-left panels). AS2COS0065.1 is shown with a limit for the lower error since it causes the error-bar to decrease below a line ratio of 1. We also show the combined composite SLED of the sample (circles), normalised by the star-formation rate surface density ($\Sigma_{\rm SFR}$) and 68\% confidence level as error-bars.
    _ Top right:_ The combined composite SLED of the sample (circles) with errors corresponding to the scatter in the sample, similar to systems such as SMM J2135-0102 (Cosmic Eyelash), NGC 253, and local ULIRGs. $J_{\rm up} = 8$ measurements are omitted for the median, since there is only one available.
    Shown alongside is the best-fit \texttt{RADEX} model to the composite CO SLED, computed from a grid of temperatures and densities, which suggests average ISM conditions of $T_{\rm kin} \sim $ 30--50 K, $\log_{10}(\langle n_{\rm H_2}\rangle[{\rm cm}^{-3}]) \sim $ 3.2--3.5, and $\langle dv/dr \rangle = $ 0.5--5 kms$^{-1}$pc$^{-1}$, though there is large covariance.
    _ Bottom left:_ Similar to the top-left panel, but fluxes are normalised to the flux of the CO(3--2) where they are available, or are otherwise normalised by the predicted flux of the CO(3--2) from the CO(4--3) using the line/brightness temperature ratio measured from \citet{Danielson_2011}.
    _ Bottom right:_ Similar to the top right panel, but with fluxes normalised to the CO(3--2).
    We add a small offset to the $J_{\rm up}$ of each measurement for each source to separate their error-bars which would otherwise overlap.
    We also show $J_{\rm up}^2$ scaling which is the expected scaling of intensities for levels in LTE and in the Rayleigh-Jeans limit.
    We compare to the CO SLEDs of other systems such as the Milky Way inner disc \citep{Fixsen_1999}, the overlap region of the antennae galaxies NGC 4038 and NGC 4039 \citep[Antenna overlap region;][]{Mashian_2015}, SMM J2135-0102 \citep[Cosmic Eyelash;][]{Danielson_2011}, M82 \citep{Weiss_2005}, NGC 253 \citep{Israel_1995,Harrison_1999,Israel_2002,Bradford_2003}, ULIRGs \citep{Papadopoulos_2012}, BzKs \citep{Daddi_2015}, and SMGs from \citet{Spilker_2014} as well as \citet{Bothwell_2013}.
     (*fig:sled*)

</div>
<div id="div_fig2">

<img src="tmp_2411.14271/./images/excitation_drivers_SigmaSFR.png" alt="Fig7" width="100%"/>

**Figure 7. -** $^{12}$CO SLED shapes of ten sources from our sample of SMGs with CO(6--5) observations, probed by measurements of their $J_{\rm up} = 3$ and $J_{\rm up} = 6$ CO line fluxes, versus star-formation rate surface density ($\Sigma_{\rm SFR}$). For sources without a CO(3--2) measurement, we find the predicted flux from the CO(4--3) measurement using the line/brightness temperature ratio measured for the archetypal SMG SMM J2135-0102 \citep{Danielson_2011}. We exclude sources AS2UDS072.0 and AS2COS0139.1, however, since the available high-$J_{\rm up}$ line was the CO(8--7) and the same excitation processes may not be present in both $J_{\rm up} = 6$ and $J_{\rm up} = 8$ regions of emission.
    We show an orthogonal distance regression (ODR) fit to our observations with the 68\% confidence level estimated by bootstrapping.
    We compare to the unresolved model from \citetalias{Narayanan_2014} with added extinction, which predicts that the increasing SLED shape correlates with sources with higher $\Sigma_{\rm SFR}$, suggesting more intense ISM properties.
    We also compare our results to those from \citet{Bournaud_2015} for simulated $z = 0$ starburst mergers and $z = 2$ discs, as well as \citet{Valentino_2020} for $z \sim$ 1.3 main-sequence galaxies and starbursts and \citet{Boogaard_2020} for $z \sim $ 1.2 ASPECS galaxies, where we converted their $R_{5,2}$ line ratios to $R_{6,3}$ using line flux ratios taken from \citet{Weiss_2005} for local starburst M82, with typical error-bars associated with the data. The fit to the low-redshift measurements was computed on the sources shown with the addition of others that had upper limits on $\Sigma_{\rm SFR}$.
    The correlation coefficient and probability for the relationship can be found in Table \ref{tab:MIC}.
    The observations of our SMGs are consistent with the \citetalias{Narayanan_2014} model to within $3\sigma$. However, we find significant scatter, indicative of intrinsic variations in the ISM for a given $\Sigma_{\rm SFR}$, which could be described by other physical properties.
    In combination with the observations from \citet{Valentino_2020}, we find that the strongest driver of CO excitation is $\Sigma_{\rm SFR}$. (*fig:model*)

</div>
<div id="div_fig3">

<img src="tmp_2411.14271/./images/excitation_drivers_U_SFE.png" alt="Fig6" width="100%"/>

**Figure 6. -** $^{12}$CO SLED shapes of ten sources from our sample of SMGs with CO(6--5) observations, probed by measurements of their $J_{\rm up} = 3$ and $J_{\rm up} = 6$ CO line fluxes, versus proposed drivers of excitation, SFE (_ left_) and $\langle U \rangle$(_ right_). For sources without a CO(3--2) measurement, we derive the predicted flux from the CO(4--3) measurement using the line/brightness temperature ratio measured for the archetypal SMG SMM J2135-0102 \citep{Danielson_2011}. We exclude sources AS2UDS072.0 and AS2COS0139.1, however, since the available high-$J_{\rm up}$ line was the CO(8--7) and the same excitation processes may not be present in both $J_{\rm up} = 6$ and $J_{\rm up} = 8$ regions of emission.
    We show orthogonal distance regression (ODR) fits to our observations with the 68\% confidence levels estimated by bootstrapping.
    We also compare our results to those from \citet{Valentino_2020} for $z \sim $ 1.3 main-sequence galaxies and starbursts, where we converted their $R_{5,2}$ line ratios to $R_{6,3}$ using line flux ratios taken from \citet{Weiss_2005} for local starburst M82, with typical error-bars associated with the data.
    Correlation coefficients and probabilities for the relationships can be found in Table \ref{tab:MIC}.
    Offsets between the line ratios of SMGs and low-redshift galaxies at fixed SFE or $\langle U\rangle$ indicate that these drivers may not be optimal to describe CO excitation, whereas models improve by considering galaxy sizes (Figure \ref{fig:model}).
     (*fig:sfe_isrf_models*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.14271"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

253  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
